In [ ]:
%load_ext autoreload
%autoreload 2

import time
import random
import pickle, os
import numpy as np
import cvxpy as cp

from copy import deepcopy

import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

from free_flyer.free_flyer import FreeFlyer
from free_flyer.utils import *

from solvers.coco_ff import CoCo_FF
from solvers.meta_ff import Meta_FF

In [ ]:
#load train/test data
prob = FreeFlyer() #use default config, pass different config file oth.
config_fn = './free_flyer/config/default.p'

config_file = open(config_fn,'rb')
dataset_name, _, _ = pickle.load(config_file); config_file.close()

relative_path = os.getcwd()
dataset_fn = relative_path + '/free_flyer/data/' + dataset_name

train_file = open(dataset_fn+'/train.p','rb')
# p_train, x_train, u_train, y_train, c_train, times_train = pickle.load(train_file)
train_data = pickle.load(train_file)
train_file.close()
x_train = train_data[1]
y_train = train_data[3]

test_file = open(dataset_fn+'/test.p','rb')
# p_test, x_test, u_test, y_test, c_test, times_test = pickle.load(test_file)
test_data = pickle.load(test_file)
p_test, x_test, u_test, y_test, c_test, times_test = test_data
test_file.close()

n_test = x_test.shape[0]

n_train = 10000
for k in train_data[0]:
    train_data[0][k] = train_data[0][k][:n_train]

### Meta-CoCo

In [ ]:
system = 'free_flyer'
prob_features = ['x0', 'obstacles_map']

max_evals = 3
meta_learner = Meta_FF(system, prob, prob_features)

n_features = 4
device_id=1
meta_learner.construct_strategies(n_features, train_data, device_id=device_id)

In [ ]:
override_model_fn = True

coco_model_fn = "MetaCoCoFF_free_flyer_20210524_1120.pt"
feas_model_fn = "MetaCoCoFF_feas_free_flyer_20210524_1120.pt"

meta_learner.load_network(coco_model_fn, override_model_fn, feas_model_fn)

In [ ]:
meta_learner.training_params['TRAINING_ITERATIONS'] = 1
meta_learner.train(train_data=train_data, verbose=True, summary_writer_fn='runs/free_flyer')

### CoCo

In [ ]:
from solvers.coco_ff import CoCo_FF

system = 'free_flyer'
prob_features = ['x0', 'obstacles_map']

max_evals = 1
mlopt_cnn = CoCo_FF(system, prob, prob_features)

n_features = 4
mlopt_cnn.construct_strategies(n_features, train_data)
mlopt_cnn.setup_network()

print(mlopt_cnn.n_strategies)

In [ ]:
mlopt_cnn.training_params['TRAINING_ITERATIONS'] = 750
mlopt_cnn.train(train_data=train_data, verbose=True)